In [ ]:
import sys
sys.path.append('../utils')
import mesh_tools as mt
from SDF_CNN import CNN_3d_multiple_split
from CNN_to_PoNQ_or_lite import CNN_to_PoNQ, CNN_to_PoNQ_large
from ABC_dataset import make_mask_close
from PoNQ import PoNQ
import mesh_tools as mt
import torch
from meshplot import plot
import igl
from tqdm import tqdm


In [ ]:
state_dict = '../../data/pretrained_PoNQ_ABC.pt'
example_mesh = '/data/nmaruani/DATASETS/fun/thai_statue.ply'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = CNN_3d_multiple_split(device=device)
model.load_state_dict(torch.load(state_dict, map_location=device))
model.to(device)
model.eval()

In [ ]:
v, f = igl.read_triangle_mesh(example_mesh)
v = 2*mt.NDCnormalize(v)
grid_n = 257
points = mt.mesh_grid(grid_n, True)
sdf = igl.signed_distance(points, v, f)[0].reshape(grid_n, grid_n, grid_n)
del v
del f
tensor_sdf = torch.tensor(sdf, dtype=torch.float32, device=device)[None, None, ...]
tensor_mask = torch.tensor(make_mask_close(sdf, grid_n), dtype=torch.bool, device=device).flatten()[None, ...]

In [ ]:
ponq2 = CNN_to_PoNQ_large(model, tensor_sdf, grid_n, tensor_mask, device=device, subd=1)

In [ ]:
mt.export_obj(*ponq2.min_cut_surface(grid_n-1), 'thai')